## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
_________________

###### Данное исследование разделим на несколько частей.

##### Часть 1. Изучение общей информации :
* [1. Изученеие файлов с данными, получение общей информации](#Part_1_step_1)

##### Часть 2. Предобработка данных:
* [2.1 Обработка пропусков](#Part_2_step_2)
* [2.2 Замена типа данных](#Part_2_step_2.2)
* [2.3 Обработка дубликатов](#Part_2_step_2.3)
* [2.4 Лемматизация](#Part_2_step_2.4)
* [2.5 Категоризация данных](#Part_2_step_2.5)

##### Часть 3. Ответы на вопросы:
* [3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?](#Part_3_step_1)
* [3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?](#Part_3_step_2)
* [3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#Part_3_step_3)
* [3.4 Как разные цели кредита влияют на его возврат в срок?](#Part_3_step_4)

#####  [Общий вывод.](#Part_final)

# Часть 1. Изучение общей информации.
<a id='Part_1_step_1'></a>
## 1. Изученеие файлов с данными, получение общей информации.

In [ ]:
import pandas as pd
from pymystem3 import Mystem
import numpy as np

In [1]:
data = pd.read_csv('/datasets/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

По сводке общей информации видно, что в количественных столбцах 'days_employed' и 'total_income' есть пропуски. В столбце 'days_employed' присутствуют отрицательные значения. В столбцах education, family_status, income_type и purpose необходимо провести леммизацию. В столбцах days_employed и total_income следует перейти к целочисленным значениям.

<a id='Part_2_step_2'></a>
# Часть 2. Предобработка данных
## 2.1 Обработка пропусков

Пропуски значений есть в двух столбцах: days_employed и total_income. 
Оценим количество пропусков к общему количеству строк.

In [3]:
'Отношение количества пропусков столбца total_income к общему количеству строк с данными: {:.0%}'.format(data['total_income'].isnull().sum() / data.shape[0] )

'Отношение количества пропусков столбца total_income к общему количеству строк с данными: 10%'

Поскольку в поставленных вопросах исследования нет связи со столбцом days_employed, то заполним его нулями.

In [4]:
data['days_employed'] = data['days_employed'].fillna(0) # убираем пустые строки в колонке days_employed

Проведем проверку корректности заполненных значений в столбце total_income, для этого проверим нет ли отрицательных или нулевых значений.

In [5]:
data['total_income'].min()

20667.26379327158

Просмотрим категории по типу занятости, оценим количество пропусков в самых крупных категориях к общему числу заполненных данных категории и посчитаем для каждой категории соотношение среднего значения к медиане, все это нужно для оценки влияния вставляемых значений на датасет.

In [6]:
income_type = data['income_type'].value_counts()
income_type

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [7]:
data['total_income'][data['income_type'] == 'сотрудник'].isnull().sum() / income_type[0] * 100 

9.9379440597176

In [8]:
data['total_income'][data['income_type'] == 'компаньон'].isnull().sum() / income_type[1] * 100 

9.99016715830875

In [9]:
data['total_income'][data['income_type'] == 'пенсионер'].isnull().sum() / income_type[2] * 100 

10.71058091286307

In [10]:
data['total_income'][data['income_type'] == 'госслужащий'].isnull().sum() / income_type[3] * 100 

10.075394105551748

Доля пропуска везде почти одинаковая и составляет около 10%. Теперь оценим соотношение среднего к медианному значению.

In [11]:
(data['total_income'][data['income_type'] == 'сотрудник'].mean() / data['total_income'][data['income_type'] == 'сотрудник'].median() -1) * 100

13.174335076145649

In [12]:
(data['total_income'][data['income_type'] == 'компаньон'].mean() / data['total_income'][data['income_type'] == 'компаньон'].median() -1) * 100

17.440164684932657

In [13]:
(data['total_income'][data['income_type'] == 'пенсионер'].mean() / data['total_income'][data['income_type'] == 'пенсионер'].median() -1) * 100

15.705235572529341

In [14]:
(data['total_income'][data['income_type'] == 'госслужащий'].mean() / data['total_income'][data['income_type'] == 'госслужащий'].median() -1) * 100

13.592991223965134

Средние значение оказывается больше медианных в диапазоне от 13% до 17%. Заменим пропуски по категориям медианным значением категории.

In [15]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

Проверим корректно ли заполнен столбец children.

In [16]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Есть отрицательные значения, скорей всего это ошибка данных или ввода, заменим их на положительные значения.
Есть и аномальное значение 20 детей, вероятно это тоже ошибка данных или ввода полученная добавлением к двойке нуля, заменим 20 на 2.

In [17]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

In [18]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

В результате предобработки данных были устранены пропуски в столбце total_income путем заполнения пустых значений медианными значениями по категориям по типу занятости income_type.
В столбце children были обнаружены и заменены отрицательные значения на положительные, а также заменено аномально высокое значение количества детей на нормальное. Вероятно, что данные ошибки появились в следствии неправильного ввода данных в соответствующие поле.

<a id='Part_2_step_2.2'></a>
## 2.2 Замена типа данных

Столбец total_income имеет тип значений float, заменим его на целочисленный тип значений int.

In [19]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Произведена замена типа данных в столбце total_income с float на int.

<a id='Part_2_step_2.3'></a>
## 2.3 Обработка дубликатов

Найдем количество дупликатов в данных и удалим их, но перед этим в столбцах со строковыми значениями перейдем к нижнему регистру.

In [20]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

In [21]:
data.duplicated().sum()

71

In [22]:
data = data.drop_duplicates()

### Вывод

Строковые данные были приведены к нижнему регистру, далее найдены и устранены 71 дублирующиеся строка.

<a id='Part_2_step_2.4'></a>
## 2.4 Лемматизация

Посмотрим какие есть уникальные цели получения кредита в столбце purpose, для этого лемматизируем его с помощью инструмента Mystem.

In [23]:
m = Mystem()
def lemma(row):
    lemmas =m.lemmatize(row)
    return lemmas
data['purpose'].apply(lemma).value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

Произведена лемматизация столбца purpose с помощью инструмента Mystem.

<a id='Part_2_step_2.5'></a>
## 2.5 Категоризация данных

Для ответов на поставленные вопросы необходимо произвести категоризацию двух стобцов: 1)purpose, 2)total_income.

#### 1) Категоризация столбца purpose.

В итоге лемматизации можно 4 целевых категории для получения кредита: 1) автомобиль, 2) свадьба, 3) недвижимость и 4) образование.
Проведем категоризацию данных и добавим в данные соответствующий столбец lem_purpose.

In [24]:
def lemma2(row):
    #row = row['purpose']
    lemmas = m.lemmatize(row)
    for word in lemmas:
        if 'жилье' in word:
            return 'недвижимость'
        if 'автомобиль' in word:
            return 'автомобиль'
        if 'свадьба' in word:
            return 'свадьба'
        if 'недвижимость' in word:
            return 'недвижимость'
        if 'образование' in word:
            return 'образование'
data['lem_purpose'] = data['purpose'].apply(lemma2)

Убедимся, что категоризация прошла корректно. Для этого визуально посмотрим на 10 строк столбцов purpose и lem_purpose, затем проверим всем строкам присвоено значения категории.

In [25]:
data[['purpose','lem_purpose']].head(10)    

,purpose,lem_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


In [26]:
data['lem_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lem_purpose, dtype: int64

In [27]:
data['lem_purpose'].value_counts().sum() == data.shape[0]

True

#### 2) Категоризация столбца total_income.

Произведем категоризацию столбца total_income, для этого сначала определим границы категорий с помощью 3-х квантилей: 25%, 50% и 75%, а затем добавим в данные соответствующий столбец income_quant.

In [28]:
quant = data['total_income'].quantile([.25, .5, .75])
quant

0.25    107623.00
0.50    142594.00
0.75    195820.25
Name: total_income, dtype: float64

Напишем функцию для категоризации со следующим условием: 1) уровень дохода ниже 25% квантиля, 2) уровень дохода от 25% до 50% квантиля, 3) уровень дохода от 50% до 75% квантиля и 4) уровень дохода свыше 75% квантиля.

In [29]:
def income_quant(row):
    if row < quant[0.25]:
        return 'До ' + str(quant[0.25])
    if row >= quant[0.25] and row < quant[0.5]:
        return 'От ' + str(quant[0.25]) + ' до ' + str(quant[0.5])
    if row >= quant[0.5] and row < quant[0.75]:
        return 'От ' + str(quant[0.5]) + ' до ' + str(quant[0.75])
    if row >= quant[0.75]:
        return 'От ' + str(quant[0.75])
data['income_quant'] = data['total_income'].apply(income_quant)
data['income_quant'].value_counts()

От 142594.0 до 195820.25    6317
До 107623.0                 5364
От 195820.25                5364
От 107623.0 до 142594.0     4409
Name: income_quant, dtype: int64

### Вывод

Для ответа на поставленные вопросы провели категоризацию 2-х столбцов: lem_purpose и total_income. В первом случае с помощью лемматизации были определены ключевые категории. Во втором случае категории были получены путем разбиение массива данных по 4-м квантилям.

<a id='Part_3_step_1'></a>
# Часть 3. Ответы на вопросы

### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос напишем функцию которая категоризирует данные по двум столбцам: children и debt. Таким образом получится 4 категории: 1) Должник с детьми, 2) Платильщик с детьми, 3) Должник без детей и 4) Платильщик без детей. Далее сделаем сводную таблицу по всем категориям.

In [30]:
def children_debt(row):
    children = row['children']
    debt = row['debt']
    if children > 0:
        if debt == 1:
            return 'Должник с детьми'
        return 'Платильщик с детьми'
    if children == 0:
        if debt == 1:
            return 'Должник без детей'
        return 'Платильщик без детей'

data['children_debt'] = data.apply(children_debt, axis = 1)
data_pivot = data.pivot_table(index = 'children_debt',  values = 'debt', aggfunc='count')
data_pivot

,debt
children_debt,
Должник без детей,1063
Должник с детьми,678
Платильщик без детей,13028
Платильщик с детьми,6685


Посчитаем процентное отношение должников к платильщикам для клиентов с детьми и без.

In [31]:
'Отношение должников к платильщикам для клиентов с детьми: {:.0%}'.format(data_pivot['debt'][1] / data_pivot['debt'][3])

'Отношение должников к платильщикам для клиентов с детьми: 10%'

In [32]:
'Отношение должников к платильщикам для клиентов без детей: {:.0%}'.format(data_pivot['debt'][0] / data_pivot['debt'][2])

'Отношение должников к платильщикам для клиентов без детей: 8%'

### Вывод

В результате анализа видно, что клиенты без детей чаще возвращают деньги в срок. Это довольно легко объясняется меньшим количестсвом трат у клиентов без детей, соответственно долг им возвращать легче.

<a id='Part_3_step_2'></a>
### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос выведем сводную таблицу, в которой покажем общее количество людей в каждой категории по семейному статусу, количество должников среди них и какую долю они составляют от общего числа.

In [33]:
data_pivot2 = data.pivot_table(index = 'family_status',  values = 'debt', aggfunc=['count', np.sum, np.mean], margins=True)
data_pivot2

,count,sum,mean
,debt,debt,debt
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
не женат / не замужем,2810,274,0.097509
All,21454,1741,0.081150


### Вывод

В результате анализа видно, что клиенты в статусе 'не женат / не замужем' и 'гражданский брак' являются лидерами в невозврате кредита, так как в данном вся долговая нагрузка идет на одно человека. 
Самыми надежными платильщиками оказались клиенты в статусе 'вдовец / вдова', это возможно связать с тем, что в данную категорию попадают "возрастные" клиенты, уровень дохода которых уже достаточно высок или они большой семьей, которая помогает с долгом.

<a id='Part_3_step_3'></a>
### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос составим сводную таблицу, в которой покажем общее количество людей в каждой категории по доходу, количество должников среди них и какую долю они составляют от общего числа.

In [34]:
data_pivot3 = data.pivot_table(index = 'income_quant',  values = 'debt', aggfunc=['count', np.sum, np.mean], margins=True)
data_pivot3

,count,sum,mean
,debt,debt,debt
income_quant,,,
До 107623.0,5364,427,0.079605
От 107623.0 до 142594.0,4409,385,0.087321
От 142594.0 до 195820.25,6317,546,0.086433
От 195820.25,5364,383,0.071402
All,21454,1741,0.081150


### Вывод

В результате анализа видно, что клиенты обладающие доходом ниже или выше среднего, являются лидерами по возврату кредита. Это можно объяснить тем, что люди имеющие доход ниже среднего стараются не увеличеть свои издержки, а люди имеющие доход выше среднего не испытывают трудности в наличии свободных средств даже при выплате долга. 
Следует отметить, что процент невозврата у клиентов без сверхдохода оказывается приблизительно на одном уровне.


<a id='Part_3_step_4'></a>
### 3.4 Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос выведем сводную таблицу, в которой покажем общее количество людей в каждой категории цели кредита, количество должников среди них и какую долю они составляют от общего числа.

In [35]:
data_pivot4 = data.pivot_table(index = 'lem_purpose',  values = 'debt', aggfunc=['count', np.sum, np.mean], margins=True)
data_pivot4

,count,sum,mean
,debt,debt,debt
lem_purpose,,,
автомобиль,4306,403,0.093590
недвижимость,10811,782,0.072334
образование,4013,370,0.092200
свадьба,2324,186,0.080034
All,21454,1741,0.081150


### Вывод

В результате анализа видно, что клиенты с целью кредита 'недвижимость' и 'свадьба' чаще возвращают кредит по сравнению с целями 'автомобиль' и 'образование'. Данное явление можно объяснить тем, что первые две категории берут молодые люди с поддержкой родственников. Задолжности по категории 'автомобиль' могут возникнуть в связи с дороговизной обслуживания авто или связоного с ним происшествием. Невозвраты по категории 'образование' могут возникнуть в связи с не трудоустройством обучающегося после окончания обучения.

<a id='Part_final'></a>
## Общий вывод

В результате анализа платежеспособности клиентов были сделаны следующие ключивые выводы:
- 1) Своевременность возврата кредита сильно зависит от наличия детей у клиента.
- 2) Своевременность возврата кредита сильно зависит от семейного статуса клиента.
- 3) Своевременность возврата кредита слабо зависит от совокупного дохода клиента.
- 4) Своевременность возврата кредита сильно зависит от цели кредита клиента.
